<a href="https://colab.research.google.com/github/marlon-br/glow-tts-colab/blob/master/glow_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%cd /content
!git clone https://github.com/NVIDIA/waveglow waveglow
%cd /content/waveglow
!gdown https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
%cd /content


In [0]:
%cd /content
!pip3 uninstall apex

!git clone https://www.github.com/nvidia/apex apex
%cd /content/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd /content

In [0]:
import torch

%cd /content/waveglow
waveglow_path = '/content/waveglow/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.cuda().eval()
%cd /content

In [0]:
from apex import amp
waveglow, _ = amp.initialize(waveglow, [], opt_level="O3") # Try if you want to boost up synthesis speed.

In [0]:
%cd /content/
!git clone https://github.com/jaywalnut310/glow-tts glow-tts

In [0]:
%cd /content/glow-tts
import utils
!mkdir /content/glow-tts/logs

!cp /content/glow-tts/configs/base.json /content/glow-tts/logs/config.json
%cd /content/glow-tts/logs
!gdown https://drive.google.com/uc?id=1JiCMBVTG4BMREK8cT3MYck1MgYvwASL0
%cd /content

In [0]:
%cd /content/glow-tts/monotonic_align/
!python setup.py install
from monotonic_align.core import maximum_path_c

In [0]:
!pip install Unidecode

In [0]:
%cd /content/glow-tts/logs

In [0]:
%cd /content/glow-tts

import models
from text import text_to_sequence, cmudict
from text.symbols import symbols
import glob
import os

model_dir = "/logs"
hps = utils.get_hparams_from_dir(model_dir)

model = models.FlowGenerator(
                              len(symbols),
                              out_channels=hps.data.n_mel_channels,
                              **hps.model).to("cuda")

f_list = glob.glob(os.path.join("/logs", "*.pth"))
#f_list.sort(key=lambda f: int("".join(filter(str.isdigit, f))))
x = f_list[-1]
print(x)

#checkpoint_path = utils.latest_checkpoint_path(model_dir)
utils.load_checkpoint(x, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

In [0]:
import numpy as np

tst_stn = " Deeeep fakes are really awesome ! " # Adding spaces at the beginning and the end of utterance improves quality
sequence = np.array(text_to_sequence(tst_stn, ['english_cleaners'], cmu_dict))[None, :]
print("".join([symbols[c] for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

In [0]:
import IPython.display as ipd

with torch.no_grad():
  noise_scale = .667
  length_scale = 1.0
  (y_gen_tst, *r), attn_gen, *_ = model(x_tst, x_tst_lengths, gen=True, noise_scale=noise_scale, length_scale=length_scale)
  try:
    audio = waveglow.infer(y_gen_tst.half(), sigma=.666)
  except:
    audio = waveglow.infer(y_gen_tst, sigma=.666)
ipd.Audio(audio[0].clamp(-1,1).data.cpu().float().numpy(), rate=hps.data.sampling_rate)